In [1]:
import pandas as pd

df = pd.read_csv("dados/amostra.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)


df["dt_despacho_pedido"] = pd.to_datetime(
    df["dt_despacho_pedido"] + " " + df["hr_despacho_pedido"],
    format="%Y-%m-%d %H:%M:%S",
)
df.drop(columns=["hr_despacho_pedido"], inplace=True)

df["dt_entrega_pedido"] = pd.to_datetime(
    df["dt_entrega_pedido"] + " " + df["hr_entrega_pedido"],
    format="%Y-%m-%d %H:%M:%S",
)
df.drop(columns=["hr_entrega_pedido"], inplace=True)

df["dt_previsao_entrega_cliente"] = pd.to_datetime(df["dt_previsao_entrega_cliente"])

df["dt_criacao"] = pd.to_datetime(df["dt_criacao"])

df["dt_pagamento_pedido"] = pd.to_datetime(df["dt_pagamento_pedido"])

df["flg_existem_ocorrencias"] = df["flg_existem_ocorrencias"].str.lower() == "sim"

df

,cod_pedido,cidade_destinatario,uf,grp_transportadora,dt_despacho_pedido,dt_entrega_pedido,dt_previsao_entrega_cliente,dt_criacao,dt_pagamento_pedido,flg_existem_ocorrencias,tp_praca,des_unidade_negocio,des_cd_origem,qtd_dias_tat,tp_performance_entrega
0,127995108-1,CURITIBA,PR,Transportadora 4,2023-12-04 05:56:39,2023-12-07 12:19:04,2023-12-07,2023-11-30,2023-11-30,False,Capital,Multi,PR-Campina G. Sul,5.0,Entregue no Prazo
1,126433711-1,PARAUAPEBAS,PA,Transportadora 2,2023-11-21 12:39:55,2023-12-01 12:23:47,2023-12-11,2023-11-15,2023-11-17,False,Interior,Mono,PR-Campina G. Sul,10.0,Entregue no Prazo
2,125177091-1,JACAREZINHO,PR,Transportadora 4,2023-10-28 10:12:24,2023-10-31 16:10:01,2023-11-06,2023-10-26,2023-10-27,False,Interior,Mono,PR-Campina G. Sul,2.0,Entregue no Prazo
3,123168303-1,SÃO PAULO,SP,Transportadora 5,2023-08-23 17:52:06,2023-08-24 12:00:13,2023-08-24,2023-08-22,2023-08-22,False,Capital,Multi,PR-Campina G. Sul,2.0,Entregue no Prazo
4,122901808,SAO LUIS,MA,Transportadora 3,2023-08-14 08:41:00,2023-08-23 15:22:59,2023-08-24,2023-08-13,2023-08-12,False,Capital,Multi,PR-Campina G. Sul,8.0,Entregue no Prazo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503527,124807656-1,NITERÓI,RJ,Transportadora 3,2023-10-16 14:59:28,2023-10-19 14:56:27,2023-10-18,2023-10-14,2023-10-14,False,Reg. Metropolitana,Mono,SP-Registro,4.0,Fora do Prazo
503528,127769981,PARNAMIRIM,RN,Transportadora 1,2023-11-27 23:28:20,NaT,2023-12-11,2023-11-26,2023-11-26,True,Interior,Mono,SP-Registro,NaN,Fora do Prazo
503529,121957602-1,Canoas,RS,Transportadora 4,2023-07-11 03:14:47,2023-07-13 18:16:59,2023-07-12,2023-07-11,2023-07-09,False,Capital,Multi,PR-Campina G. Sul,4.0,Fora do Prazo
503530,123101228-1,GUARULHOS,SP,Transportadora 1,NaT,2023-08-24 11:13:49,2023-08-23,2023-08-19,2023-08-19,False,Reg. Metropolitana,Mono,SP-Registro,4.0,Fora do Prazo
